In [ ]:
file_path='ai_agents/walmart_sales.xlsx'

In [5]:
from langchain.agents import Tool
import pandas as pd

def read_excel(query: str):
    df = pd.read_excel(file_path, sheet_name="SalesData")
    if "total sales" in query.lower():
        return f"Total sales: ₹{df['Sales'].sum()}"
    elif "store" in query.lower():
        return df.groupby("Store")["Sales"].sum().to_string()
    else:
        return df.head().to_string()

excel_tool = Tool(
    name="ExcelReader",
    func=read_excel,
    description="Use to query walmart_sales.xlsx based on user input"
)

In [6]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0.3)

agent = initialize_agent(
    tools=[excel_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Show total sales and breakdown by store.")




> Entering new AgentExecutor chain...
 I should use the ExcelReader to query the walmart_sales.xlsx file.
Action: ExcelReader
Action Input: "SELECT Store, SUM(Sales) FROM walmart_sales.xlsx GROUP BY Store"
Observation: Store
Store_A     9519
Store_B    15230
Store_C     4912
Store_D     7467
Thought: I should add up the total sales for all stores.
Action: ExcelReader
Action Input: "SELECT SUM(Sales) FROM walmart_sales.xlsx"
Observation:          Date  Product     Category  Sales    Store
0  2024-01-01   Laptop  Electronics   1200  Store_A
1  2024-01-02       TV  Electronics   3000  Store_B
2  2024-01-02  Shampoo       Health    200  Store_A
3  2024-01-03     Sofa    Furniture   4932  Store_D
4  2024-01-04     Sofa    Furniture   4974  Store_A
Thought: I now know the final answer.
Final Answer: The total sales for all stores is $29,813.

> Finished chain.


'The total sales for all stores is $29,813.'

In [9]:
from pprint import pprint
#Step 5: Generate Report from Prompt
summary_prompt = f"""
You are a Walmart reporting assistant. Create a business summary based on this:
{read_excel("summary")}
"""

summary = llm.invoke(summary_prompt)
pprint(summary)

('\n'
 'Business Summary for Walmart:\n'
 '\n'
 'During the first week of 2024, Walmart saw a total sales of $9,306 across '
 'its stores. Electronics was the top selling category, with a total of $4,200 '
 'in sales, followed by Furniture with $9,906 in sales. Store_A had the '
 'highest sales with a total of $6,374, followed by Store_B with $3,000 and '
 'Store_D with $4,932. The top selling product was the Sofa, with a total of '
 '$9,906 in sales, followed by the Laptop with $1,200 and the TV with $3,000. '
 'The Health category had the lowest sales with a total of $200. Overall, '
 'Walmart had a successful first week of 2024, with strong sales in the '
 'Electronics and Furniture categories.')


In [ ]:
cluster_endpoint = ""
api_key = "API_KEY"

In [11]:
from pymilvus import connections

connections.connect(
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)

print("✅ Connected to Milvus Cloud!")

✅ Connected to Milvus Cloud!


In [28]:
from pymilvus import connections, Collection
from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType
from pymilvus import Index

# Connect to Milvus (mock or real)

connections.connect(
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)

# Embed and store past report
model = SentenceTransformer("all-MiniLM-L6-v2")

text = "Store_A had highest sales last month. Recommend restocking."
embedding = model.encode(text).tolist()

# Define collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="Date", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="Product", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="Category", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="Sales", dtype=DataType.FLOAT),
    FieldSchema(name="Store", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384)
]

schema = CollectionSchema(fields, description="Walmart sales with semantic search")
collection = Collection(name="walmart_sales1", schema=schema)

# Insert initial data into collectio
model = SentenceTransformer("all-MiniLM-L6-v2")
text = "Store_A had highest sales last month. Recommend restocking."
embedding = model.encode(text).tolist()

# Insert into collection (column-based format)
collection.insert([
    [1],                         # id
    ["2023-10-01"],              # Date
    ["Laptop"],                  # Product
    ["Electronics"],            # Category
    [1200.0],                    # Sales
    ["Store_A"],                 # Store
    [embedding]                  # Embedding
])

schema = CollectionSchema(fields)

collection = Collection("walmart_sales2", schema)

# Insert into Milvus (use existing collection schema)
# Retrieve similar reports on demand using cosine similarity

# Create index for efficient search

index = Index(
    collection=collection,
    field_name="embedding",
    index_params={"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 128}}
)
# Load the collection
collection.load()
# Query similar reports
query_embedding = model.encode("What were the sales trends last month?").tolist()

results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["Date", "Product", "Category", "Sales", "Store"]
)
# Display results
for result in results[0]:
    print(f"Date: {result.entity.get('Date')}, Product: {result.entity.get('Product')}, "
          f"Category: {result.entity.get('Category')}, Sales: {result.entity.get('Sales')}, "
          f"Store: {result.entity.get('Store')}")
# Cleanup
collection.release()
# Disconnect from Milvus

In [30]:
from pymilvus import Index

index = Index(
    collection=collection,
    field_name="embedding",
    index_params={"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 128}}
)
collection.load()

In [31]:
query_embedding = model.encode("What were the sales trends last month?").tolist()

results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["Date", "Product", "Category", "Sales", "Store"]
)

# Display results
for result in results[0]:
    print(f"Score: {result.score:.2f}")
    print(f"Product: {result.entity.get('Product')}, Store: {result.entity.get('Store')}, Sales: {result.entity.get('Sales')}")

In [35]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, Index
from sentence_transformers import SentenceTransformer

# ✅ Connect to Milvus
connections.connect(
    alias="default",
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)

# ✅ Load Sentence Transformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ Create collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="Date", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="Product", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="Category", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="Sales", dtype=DataType.FLOAT),
    FieldSchema(name="Store", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384)
]

schema = CollectionSchema(fields, description="Walmart sales with semantic search")

# ✅ Create or get collection
collection_name = "walmart_sales"

from pymilvus import utility

if utility.has_collection(collection_name):
    Collection(collection_name).drop()

collection = Collection(name=collection_name, schema=schema)


# ✅ Insert data
text = "Store_A had highest sales last month. Recommend restocking."
embedding = model.encode(text).tolist()

collection.insert([
    [1],                         # id
    ["2023-10-01"],              # Date
    ["Laptop"],                  # Product
    ["Electronics"],             # Category
    [1200.0],                    # Sales
    ["Store_A"],                 # Store
    [embedding]                  # Embedding
])

# ✅ Create vector index
index = Index(
    collection=collection,
    field_name="embedding",
    index_params={"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 128}}
)

# ✅ Load collection
collection.load()

# ✅ Run semantic search
query_embedding = model.encode("What were the sales trends last month?").tolist()
results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["Date", "Product", "Category", "Sales", "Store"]
)

# ✅ Display results
print("\n🔍 Search Results:")
for result in results[0]:
    print(f"Score: {result.score:.3f} | Date: {result.entity.get('Date')}, "
          f"Product: {result.entity.get('Product')}, Category: {result.entity.get('Category')}, "
          f"Sales: {result.entity.get('Sales')}, Store: {result.entity.get('Store')}")

# ✅ Clean up
collection.release()
connections.disconnect(alias="default")


🔍 Search Results:
